In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = 4

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows={
            'train': {
                1: ("2014-01-01", "2014-02-01")
            },
            'unseen_activations_of_seen_appliances': {
                1: ("2014-02-02", "2014-02-08")                
            },
            'unseen_appliances': {
                2: ("2013-06-01", "2013-06-07")
            }
        }
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 112 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 119 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 30 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [3]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [4]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False
)

In [ ]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [ ]:
all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [ ]:
fig, axes = plt.subplots(2, sharex=True)
seq.all_appliances.plot(ax=axes[0])
axes[1].plot(seq.input)
fig.suptitle(FOLD)
plt.show()

In [5]:
sample = source.get_batch(num_seq_per_batch=1024)

In [6]:
pipeline = DataPipeline(
    [source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std())],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

In [ ]:
nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

In [ ]:
disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

In [ ]:
disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [7]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
   'allow_incomplete_target': False,
   'distractor_inclusion_prob': 1.0,
   'include_incomplete_target_in_output': True,
   'rng_seed': None,
   'seq_length': 256,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 1.0,
   'uniform_prob_of_selecting_each_model': True},
  'input_processing': [{'DivideBy': {'divisor': 981.0473}}],
  'num_seq_per_batch': 64,
  'target_processing': [{'DivideBy': {'divisor': 604.02991}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}}}}

In [8]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [9]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [10]:
from pymongo import MongoClient
client = MongoClient()
client.drop_database('neuralnilm_experiments')
db = client.neuralnilm_experiments

In [12]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    db=db,
    experiment_id=0,
    metrics=Metrics([4]),
    learning_rate=1E-2,
    repeat_callbacks=[
        ( 100, Trainer.validate)
    ]
)

INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02


In [13]:
trainer.fit(101)

INFO:neuralnilm.trainer:Starting training for 101 iterations.
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.
INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Stopped training. Completed 101 iterations.


In [14]:
trainer.iteration

101

In [17]:
import pymongo
list(db.train_scores.find().sort('iteration', pymongo.DESCENDING).limit(50))

[{u'_id': ObjectId('55db1d35a0777c27beb8853a'),
  u'experiment_id': 0,
  u'iteration': 100,
  u'loss': 0.9001259207725525,
  u'source_id': 0},
 {u'_id': ObjectId('55db1d35a0777c27beb88539'),
  u'experiment_id': 0,
  u'iteration': 99,
  u'loss': 0.9855844378471375,
  u'source_id': 0},
 {u'_id': ObjectId('55db1d35a0777c27beb88538'),
  u'experiment_id': 0,
  u'iteration': 98,
  u'loss': 0.8240548968315125,
  u'source_id': 0},
 {u'_id': ObjectId('55db1d35a0777c27beb88537'),
  u'experiment_id': 0,
  u'iteration': 97,
  u'loss': 0.956082820892334,
  u'source_id': 0},
 {u'_id': ObjectId('55db1d35a0777c27beb88536'),
  u'experiment_id': 0,
  u'iteration': 96,
  u'loss': 0.996891975402832,
  u'source_id': 0},
 {u'_id': ObjectId('55db1d35a0777c27beb88535'),
  u'experiment_id': 0,
  u'iteration': 95,
  u'loss': 1.0046125650405884,
  u'source_id': 0},
 {u'_id': ObjectId('55db1d35a0777c27beb88534'),
  u'experiment_id': 0,
  u'iteration': 94,
  u'loss': 1.0917679071426392,
  u'source_id': 0},
 {u'_id

In [18]:
list(db.validation_scores.find().sort('iteration', pymongo.DESCENDING).limit(50))

[{u'SyntheticAggregateSource': {u'train': {u'2_state_classification': {u'accuracy_score': 0.9979248046875,
     u'f1_score': 0.0,
     u'precision_score': 0.0,
     u'recall_score': 0.0},
    u'regression': {u'mean_absolute_error': 0.008479020558297634,
     u'relative_error_in_total_energy': -1.0}},
   u'unseen_activations_of_seen_appliances': {u'2_state_classification': {u'accuracy_score': 0.99859619140625,
     u'f1_score': 0.0,
     u'precision_score': 0.0,
     u'recall_score': 0.0},
    u'regression': {u'mean_absolute_error': 0.00579441525042057,
     u'relative_error_in_total_energy': -1.0}},
   u'unseen_appliances': {u'2_state_classification': {u'accuracy_score': 0.88494873046875,
     u'f1_score': 0.0,
     u'precision_score': 0.0,
     u'recall_score': 0.0},
    u'regression': {u'mean_absolute_error': 0.5612585544586182,
     u'relative_error_in_total_energy': -1.0}}},
  u'_id': ObjectId('55db1d36a0777c27beb8853b'),
  u'experiment_id': 0,
  u'iteration': 100},
 {u'SyntheticAg